## Q1.

Add methods `__iter__` to your project Time Series class to iterate over values, a method `itertimes` to iterate over times, a method `itervalues` to iterate over values, and a method `iteritems` to iterate over time-value pairs. (This is a similar interface to python dictionaries). To test these, check both the types of the results and the answers you expect.

In [1]:
#your code here

# PROJECT WORK, DONE IN TIMESERIES

## Q2.

An online mean and standard deviation algorithm.

Below is a function to generate a potentially infinite stream of 1-D data.

In [2]:
from random import normalvariate, random
from itertools import count
def make_data(m, stop=None):
    for _ in count():
        if stop and _ > stop:
            break
        yield 1.0e09 + normalvariate(0, m*random() )
        

Here is an implementation of an online mean algorithm..see http://www.johndcook.com/blog/standard_deviation/ and the link to http://www.johndcook.com/blog/2008/09/26/comparing-three-methods-of-computing-standard-deviation/ in-between. (Convince yourselves of the formulas...)

In [3]:
def online_mean(iterator):
    n = 0
    mu = 0
    for value in iterator:
        n += 1
        delta = value - mu
        mu = mu + delta/n
        yield mu

We use out generator functions to implement iterators:

In [4]:
g = make_data(5, 10)
list(g)

[999999997.6335508,
 999999995.5217751,
 1000000007.2486397,
 1000000005.230493,
 999999999.725441,
 999999998.2443409,
 999999999.4318136,
 1000000000.9916475,
 1000000004.1567211,
 999999999.4261043,
 1000000000.2778298]

In [5]:
g = online_mean(make_data(5, 100))
print(type(g))
list(g)

<class 'generator'>


[1000000003.2361003,
 1000000001.094898,
 1000000000.6241463,
 1000000000.0102983,
 1000000000.3717247,
 1000000000.2115808,
 1000000000.2315459,
 1000000000.6636621,
 1000000000.7987757,
 1000000000.2435302,
 1000000000.2292694,
 1000000000.0952154,
 1000000000.1129824,
 1000000000.5425407,
 1000000000.353186,
 1000000000.2868539,
 1000000000.2334214,
 1000000000.2709838,
 1000000000.1792333,
 1000000000.2532129,
 1000000000.1851764,
 1000000000.2313493,
 1000000000.1107204,
 1000000000.2229116,
 1000000000.1449037,
 1000000000.0775439,
 1000000000.1414028,
 1000000000.1454087,
 999999999.9856699,
 999999999.9089572,
 999999999.843495,
 999999999.8356439,
 999999999.8434027,
 999999999.8842756,
 999999999.9636945,
 1000000000.047346,
 999999999.76685,
 999999999.7731135,
 999999999.8477881,
 1000000000.0522307,
 1000000000.0443356,
 1000000000.0228616,
 999999999.9464047,
 1000000000.033765,
 1000000000.0067601,
 999999999.9884827,
 999999999.9882652,
 1000000000.0689893,
 1000000000.

### 2.1

Implement the standard deviation algorithm as a generator function as

```python
def online_mean_dev(iterator):
    BLA BLA
    if n > 1:
        stddev = math.sqrt(dev_accum/(n-1))
        yield (n, value, mu, stddev)
```

In [14]:
# your code here
import math
# your code here
def online_mean_dev(iterator):
    n=0
    mu=0
    std=0
    for value in iterator:
        n+= 1
        mu_in= mu
        delta= value - mu
        mu = mu + delta/n
        
        if n > 1:
            error = std + (value - mu_in) * (value - mu_in)
            std = math.sqrt(error/(n-1))
            yield (n, value, mu, std)

Here we make 100000 element data, and run this iterator on it (imagine running this on a time-series being slowly read from disk

In [7]:
data_with_stats = online_mean_dev(make_data(5, 100000))

## Q3.

Let's do Anomaly detection. Write a routine `is_ok`:

```python
def is_ok(level, t)
```

which takes a tuple like the one yielded by your code above and returns True if the value is inbetween `level`-$\sigma$ of the mean.

In [13]:
#your code here
def is_ok(level, t):
    value= t[1]
    mu= t[2]
    std= t[3]
    diff= level-std
    if diff < 0:
        diff = 0
    high= mu+diff
    low= mu-diff
    if (value>=low) and (value<=high):
        yield True
    yield False


We use this function to create a predicate passed through to `itertools.filterfalse` which is then used to obtain an iterator on the anomalies.

In [9]:
from itertools import filterfalse
pred = lambda t: is_ok(5, t)
anomalies = filterfalse(pred, data_with_stats)

We materialize the anomalies...

In [10]:
list(anomalies)#materialize

[]

## To think of, but not hand in

What kinds of anomalies will this algorithm pick up? What kinds would a shorter "window" of anomaly detection, like 100 points around the time in question pick? How might you create an algorithm which does window based averaging? (hint: the window size is small compared to the time series size). 

Finally think a bit of how you might implement all of this in a production environment..remember that data streaming in might get backed up when you handle an anomaly.

(Some inspiration might accrue if you look at the docs for `collections.deque`).